In [1]:
from data.CustomImageDataset import GenerationImageDataset
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

## Chargement des données

In [2]:
dataset_path = './data/datasets/satellite_image_caption_generation'

transformations = transforms.Compose([
    transforms.Grayscale(1)
])

train_dataset = GenerationImageDataset(dataset_path=dataset_path, annotations_file="train.csv", transform=transformations)
validation_dataset = GenerationImageDataset(dataset_path=dataset_path, annotations_file="valid.csv", transform=transformations)
test_dataset = GenerationImageDataset(dataset_path=dataset_path, annotations_file="test.csv", transform=transformations)

In [3]:
print(f'taille dataset d\'entrainement {len(train_dataset)}')
print(f'taille dataset de validation {len(validation_dataset)}')
print(f'taille dataset de test {len(test_dataset)}')

taille dataset d'entrainement 8734
taille dataset de validation 1094
taille dataset de test 1093


## Affichage de quelques données

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=3)
plt.axis('off')
for i in range(3):
    for j in range(3):
        axs[i, j].axis('off')
        axs[i, j].imshow(train_dataset[3*i + j].permute(1,2,0))

## Entrainement

In [4]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from utils.trainer.GenerationTrainer import GenerationTrainer
from models.UNet import UNet

learning_rate = 0.05
model = UNet(in_channels=1, depth=5, num_classes=256, task_name='generation', dataset_name='satellite_image_caption')

loss = CrossEntropyLoss()
optimizer  = Adam(params=model.parameters(), lr=learning_rate)

trainer = GenerationTrainer(model=model, train_dataset=train_dataset, test_dataset=test_dataset, loss_fn=loss, optimizer=optimizer, batch_size=24)

device: cpu


In [ ]:
trainer.train(num_epochs=1)

## Affiche les infos sur le modèles

In [5]:
from torchinfo import summary

summary_kwargs = dict(col_names=['input_size', 'output_size', 'kernel_size', 'num_params', 'mult_adds'], depth=3, verbose=0)

summary(model, input_size=train_dataset[0].shape, batch_dim=0, **summary_kwargs)

Layer (type:depth-idx)                        Input Shape               Output Shape              Kernel Shape              Param #                   Mult-Adds
UNet                                          [1, 1, 224, 224]          [1, 256, 224, 224]        --                        --                        --
├─ModuleDict: 1-1                             --                        --                        --                        --                        --
│    └─ResidualBlock: 2-1                     [1, 1, 224, 224]          [1, 64, 224, 224]         --                        --                        --
│    │    └─Conv2d: 3-1                       [1, 1, 224, 224]          [1, 64, 224, 224]         [3, 3]                    576                       28,901,376
│    │    └─BatchNorm2d: 3-2                  [1, 64, 224, 224]         [1, 64, 224, 224]         --                        128                       128
│    │    └─Conv2d: 3-3                       [1, 64, 224, 224]   

Rappel, une RTX 4080 à une puissance de 48.74 TFlops (FP32) Soit $48.74 \times 10^{12}$ opérations par secondes en float32.
Avec une profondeur de 5 blocs:
Ici, nous lisons que le modèle effectue $82.15 \times 10^9$ opérations pour une image avec des couches résiduelles. Nous avons aussi 114 millions de paramètres au total.
En utilisant remplaçant les couches résiduelles par des ResidualBottleneck, nous obtenons $20.26 \times 10^9$ opérations et 33 millions de paramètres pour le modèle.